In [1]:
# Import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import findspark
findspark.init()

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
import pyspark.pandas as ps

# create the Spark Session
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

# # create the Spark Context
# sc = spark.sparkContext

c:\Users\Wenhao\miniconda3\envs\cse-547\Lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [ ]:
# friend = spark.read.csv("hw1-bundle/hw1-bundle/q1/data/soc-LiveJournal1Adj.txt", sep='\t')
# friend = friend.select(col("_c0").alias("User"), col("_c1").alias("Friends"))
# friend = friend.withColumn('Friends', split('Friends', ','))
# friend.show(5)

In [3]:
friend = spark.read.csv("hw1-bundle/hw1-bundle/q1/data/soc-LiveJournal1Adj.txt", sep='\t')
friend = friend.toPandas()
friend = friend.rename({'_c0': 'User', '_c1': 'Friends'}, axis = 1)
friend['User'] = friend['User'].astype(int)
friend['Friends'] = friend['Friends'].apply(lambda x: set([]) if (x is None) else set(x.split(','))) # split the index of friends into list
no_friend = friend[friend['Friends'].apply(len) == 0].rename({'Friends': 'Recommendations'}, axis = 1) # filter users who have no friend
friend = friend[friend['Friends'].apply(len) > 0] # filter users who have at least one friend
friend = friend.iloc[:100,] # FIX ME: SUBSAMPLING
friend

,User,Friends
0,0,"{36, 76, 30, 49, 66, 64, 91, 27, 71, 43, 72, 2..."
1,1,"{20075, 12347, 35678, 29724, 13845, 49226, 344..."
2,2,"{16659, 45046, 117, 49927, 135, 24539, 0, 2755..."
3,3,"{0, 13185, 12, 12636, 55, 27552, 38737, 1532, 41}"
4,4,"{8, 14, 19079, 42697, 38792, 38822, 77, 27, 72..."
...,...,...
95,12636,"{12670, 12678, 3, 12649, 6027, 36395, 35851, 9..."
96,13185,"{40812, 17847, 13183, 3, 41436, 30724, 49560, ..."
97,27552,"{37466, 37034, 27584, 27549, 27626, 27585, 275..."
98,72,"{8, 8136, 0, 125, 4}"


In [4]:
no_friend

,User,Recommendations
1598,43159,{}
1635,43198,{}
1669,43234,{}
1696,43261,{}
1697,43263,{}
...,...,...
49909,49773,{}
49990,49991,{}
49991,49993,{}
49993,49996,{}


In [5]:
all_user_id = set(friend['User'].unique())
unfriend = friend.copy()
unfriend['Unfriends'] = friend['Friends'].apply(lambda x: all_user_id - x)
unfriend = unfriend.drop('Friends', axis=1)
unfriend = unfriend.explode('Unfriends')
unfriend['Unfriends'] = unfriend['Unfriends'].astype(int)
unfriend['Mutual Friends'] = unfriend.apply(lambda row: 
                                            (friend[friend['User'] == row['User']]['Friends'].iloc[0]) & 
                                            (friend[friend['User'] == row['Unfriends']]['Friends'].iloc[0]), axis=1)
unfriend['Num of Mutual Friends'] = unfriend['Mutual Friends'].apply(len)
unfriend.head()

,User,Unfriends
0,0,0
0,0,1
0,0,2
0,0,3
0,0,4


In [20]:
unfriend['Mutual Friends'] = unfriend.apply(lambda row: (friend[friend['User'] == row['User']]['Friends'].iloc[0]) & (friend[friend['User'] == row['Unfriends']]['Friends'].iloc[0]), axis=1)

In [21]:
unfriend['Num of Mutual Friends'] = unfriend['Mutual Friends'].apply(len)

In [22]:
unfriend

,User,Unfriends,Mutual Friends,Num of Mutual Friends
0,0,0,"{50, 82, 76, 7, 72, 63, 48, 57, 75, 53, 45, 32...",94
0,0,1,"{5, 20}",2
0,0,2,{},0
0,0,3,"{12, 55, 41}",3
0,0,4,"{8, 72, 74, 77, 15, 18, 14, 80, 27}",9
...,...,...,...,...
99,80,32489,{},0
99,80,35613,{},0
99,80,41456,{},0
99,80,45046,{},0


In [23]:
mutual_friend = unfriend.sort_values(['User', 'Num of Mutual Friends', 'Unfriends'], ascending = False).groupby('User').head(10)

In [24]:
mutual_friend.head(20)

,User,Unfriends,Mutual Friends,Num of Mutual Friends
83,49985,49985,"{35565, 38737, 33072, 41442, 25228, 23512, 499...",55
83,49985,35678,"{14112, 35630, 25228, 49917, 1, 21556, 23690, ...",8
83,49985,35633,"{35630, 41433, 25228, 1, 21556, 35562, 35621, ...",8
83,49985,35613,"{35630, 38737, 33072, 25228, 1, 21556, 23690, ...",8
83,49985,35605,"{35630, 25228, 49917, 1, 35562, 35621}",6
83,49985,23520,"{27536, 35565, 41433, 1, 19138, 35621}",6
83,49985,21556,"{35630, 25228, 1, 49546, 23690, 35621}",6
83,49985,20075,"{25228, 1, 21556, 35630, 35621}",5
83,49985,1,"{27536, 38737, 31232, 21556, 34406}",5
83,49985,49226,"{27536, 35630, 35621, 1}",4


In [25]:
len(unfriend['User'].unique())

100

In [12]:
friend.apply(lambda row: row['Friends'] & friend[friend]['Friends'], axis=1)

,User,Friends,Unfriends
0,0,"{13, 25, 35, 37, 94, 60, 17, 6, 11, 77, 5, 34,...",12636
0,0,"{13, 25, 35, 37, 94, 60, 17, 6, 11, 77, 5, 34,...",34394
0,0,"{13, 25, 35, 37, 94, 60, 17, 6, 11, 77, 5, 34,...",44887
0,0,"{13, 25, 35, 37, 94, 60, 17, 6, 11, 77, 5, 34,...",35648
0,0,"{13, 25, 35, 37, 94, 60, 17, 6, 11, 77, 5, 34,...",43447
...,...,...,...
99,80,"{15, 4, 125, 0}",12846
99,80,"{15, 4, 125, 0}",40
99,80,"{15, 4, 125, 0}",55
99,80,"{15, 4, 125, 0}",13840


In [5]:
# friend['Friends'].apply(lambda x: [y for y in all_user_id if y not in x])

KeyboardInterrupt: 

In [25]:
set(all_user_id) - set(friend['Friends'])

TypeError: unhashable type: 'list'

In [17]:
all_user_id

array(['0', '1', '2', ..., '49994', '49996', '49999'], dtype=object)

In [13]:
len(friend)

49995

In [24]:
split_by_comma = lambda x: x.split(',')
split_by_comma = udf(split_by_comma, StringType())

In [35]:
all_user_id = set(range(friend.count()))

In [38]:
unfriend = friend.withColumn('Friends', all_user_id - set(split('Friends', ',')))

PySparkTypeError: [NOT_ITERABLE] Column is not iterable.

In [30]:
friend.show(5)

+----+--------------------+
|User|             Friends|
+----+--------------------+
|   0|[1, 2, 3, 4, 5, 6...|
|   1|[0, 5, 20, 135, 2...|
|   2|[0, 117, 135, 122...|
|   3|[0, 12, 41, 55, 1...|
|   4|[0, 8, 14, 15, 18...|
+----+--------------------+
only showing top 5 rows



In [8]:
friend.select("Friends").take(10)

[Row(Friends='1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94'),
 Row(Friends='0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592'),
 Row(Friends='0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878'),
 Row(Friends='0,12,41,55,1532,12636,13185,27552,38737'),
 Row(Friends='0,8,14,15,18,27,72,80,15326,19068,19079,24596,42697,46126,74,77,33269,38792,38822'),
 Row(Friends='0,1,20,2022,22939,23527,30257,32503,35633

In [ ]:
unfriend = 

In [2]:
findspark.find()

'c:\\Users\\Wenhao\\miniconda3\\envs\\cse-547\\Lib\\site-packages\\pyspark'